In [ ]:
%cd ..
%pip install -e .

In [ ]:
import random

import numpy as np
import pandas as pd

from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN0
from src.models.gat import GAT

from src.models.train_model import train_with_loss
from src.models.train_model import random_splits

from src.models.reg import make_preg_loss
from src.models.reg import make_lap_loss

from src.models.reg import compute_a_hat

from src.models.evaluate_model import evaluate0
from src.models.evaluate_model import evaluate1

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from sklearn.manifold import TSNE

from src.models.evaluate_model import test

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0].to(device)

L_cls = lambda x, y: F.nll_loss(torch.log(x), y, reduction='sum')
L_preg = lambda x, y: F.cross_entropy(x, y, reduction='sum')

A_hat = compute_a_hat(data)
mu = 1

loss_fn = make_preg_loss(L_cls, L_preg, mu, A_hat)

metrics = []
for mu in range(11):
    mu = mu / 10
    # for num_training_nodes in  tqdm([20, 40, 60, 80, 100]):

    torch.manual_seed(1)
    random.seed(1)

    model = GCN0(num_node_features=dataset.num_node_features,
                num_classes=dataset.num_classes) \
                .to(device)

    model = train_with_loss(model, data, loss_fn, num_epochs=200)
    # gcn_model = train2(gcn_model, data)
    acc = evaluate0(model, data)
    # print(f'mu: {mu}, reg, Accuracy: {acc:.4f}')

    train_acc, val_acc, test_acc = evaluate1(model, data)
    metrics.append({'model': model_name, 'unmask preg ratio': alpha, 'train_acc': train_acc, 'val_acc': val_acc, 'test_acc': test_acc})



In [ ]:
df = pd.DataFrame(metrics)
fig, ax = plt.subplots()

filt = (df['model'] == 'gcn')
ax.plot(df[filt]['unmask preg ratio'], df[filt]['test_acc'], '-*b')

filt = (df['model'] == 'gat')
ax.plot(df[filt]['unmask preg ratio'], df[filt]['test_acc'], '-*r')

ax.set(ylim=(.8, .83))
plt.show()